In [73]:
# some useful mysklearn package import statements and reloads
import importlib

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation


In [74]:
labels = ["True","False"]
space_table = MyPyTable().load_from_file('input_data/space.csv')

space_table.remove_rows_with_missing_values()

space_dict = myutils.count_label_occurrences(space_table.get_column("is_hazardous"))
for value in sorted(space_dict.keys()):
    print(value,space_dict[value])
    pass

discretized_label_dict = myutils.preprocess_table(space_table)
print(discretized_label_dict)
y_true = space_table.get_column(space_table.column_names.index("is_hazardous"))  
X_train = [row[:space_table.column_names.index("is_hazardous")] + row[space_table.column_names.index("is_hazardous") + 1:] for row in space_table.data]

tree_classifier = MyDecisionTreeClassifier()
y_pred = myevaluation.cross_val_predict(tree_classifier,X_train,y_true,10)
print(myevaluation.pseudo_classification_report(y_true,y_pred,labels))
# print("Decision Tree Structure:", tree_classifier.tree)
 
header = space_table.column_names
tree_classifier.print_decision_rules(attribute_names=header)
tree_classifier.visualize_tree(
    "space_tree.dot", "space_tree", attribute_names=header
)


False 295009
True 43162
0.1272198785
0.2844722965
54386.6384126095
55555809.48674465
False
unclean data 1.1029478453
clean 1.1029478453
unclean data 2.4662663578
clean 2.4662663578
unclean data 138171.4208721785
clean 138171.4208721785
unclean data 74715777.37377247
clean 74715777.37377247
{'min_labels': ['0.0 to 0.1', '0.1 to 1.1'], 'max_labels': ['0.0 to 0.3', '0.3 to 2.5'], 'velocity_labels': ['0.0 to 53433.8', '53433.8 to 138171.4'], 'miss_labels': ['0.0 to 43869660.1', '43869660.1 to 74715777.4']}
Decision Tree Classifier: Accuracy = 0.74, Error Rate = 0.26, Precision = 0.7159, Recall = 0.8040, F1 Score = 0.7574
╒═════════════════╤════════╤═════════╕
│ Is Hazardous:   │   True │   False │
╞═════════════════╪════════╪═════════╡
│ True            │    804 │     196 │
├─────────────────┼────────┼─────────┤
│ False           │    319 │     681 │
╘═════════════════╧════════╧═════════╛
IF estimated_diameter_min == 0 AND relative_velocity == 0 AND miss_distance == 0 THEN class = False.
I